In [1]:
import os
os.chdir("/Users/n0362948/desktop/Python Process Input Files/Input files old")

In [2]:
import sqlite3
import pandas as pd
all_qtrs_YTD=['2019Q1','2019Q2']    ### ['2019Q1'] if Q1;['2019Q1','2019Q2'] if Q2 ; ['2019Q1', '2019Q2','2019Q3'] if Q3 etc... 
current_qtr='2019Q2'          ## define this first and change the next two accordingly 
AsAtPeriod=201906           ##always change this the last month of curent quarter. 2019Q1-->201903    
PriorQuarter = 20081   ##always change this to 11 years and 1 quarter before current. 2019Q1-->20074

#### need to save the finance data as csv if it is in xlsb format
##########
####  D.Add a column to the left of ‘SAPLOBCode.’ Give new column same name, ‘SAPLOBCode.’
####  a.In D2, type formula, =IF(ISNUMBER(E2),E2,RIGHT(F2,5)) & drag down.
####  b.Copy and paste as values column D.
####  c.Delete Column E.
####  d.Save file as csv.
########### the read_csv line below reads in the finance data. change to a proper directory each time. 

finance_raw=pd.read_csv('/Users/n0362948/desktop/Python Process Input Files/finance data to read to python/Actuarial_Finance_Data_Reconciliation_2019Q2_QTD_v09Jul2019.xlsb.csv',encoding = "ISO-8859-1") 
SAP_Company_Map_df=pd.read_excel('/Users/n0362948/desktop/Python Process Input Files/Input files old/SAP_Company_Map.xlsx')

###IMPORTANT FOR ACTUARIAL EXHIBITS####
###### T:\Global\Reserving\20YYQQ\Data\ADSA\Import Files ######
#### looks like the ResQ_Prem_Load and ResQ_Loss_Load files should have all the data you need for that step in the import files folder. Also, it seems that Accident health is saved separately in the import files with an “AH” in the title. When you copy over the files you can copy and paste the Accident health data to the bottom of the main files.


ResQ_Loss_Load_df=pd.read_csv('/Users/n0362948/desktop/Python Process Input Files/Input files old/ResQ_Loss_Load.csv')
ResQ_Prem_Load_df=pd.read_csv('/Users/n0362948/desktop/Python Process Input Files/Input files old/ResQ_Prem_Load.csv')

output_file_Exhibits='/Users/n0362948/desktop/Python Process Input Files/output files old/Exhibits_df.xlsx'
output_finance_raw='/Users/n0362948/desktop/Python Process Input Files/output files old/Financials.xlsx'
output_financial_OrigCurr='/Users/n0362948/desktop/Python Process Input Files/output files old/Financials_OrigCurr.xlsx'
output_combined_df='/Users/n0362948/desktop/Python Process Input Files/output files old/Actuarial.xlsx'



Source_entity_mapping={'AU (AUD)':'AU',
'CBC (USD)':'CA',    
'CBC (CAD)':'CA',
'MEC': 'US',
'EU':'EU',
'HK':'HK',
'MA':'MA',
'SP':'SP',
'US':'US',
'AU':'AU' }


parameter_loss=(PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter)
parameter_prem=(PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter)

sqlite_file = '/Users/n0362948/desktop/Python Process Input Files/my_db.sqlite'
conn = sqlite3.connect(sqlite_file)

In [3]:
for filename in os.listdir('/Users/n0362948/desktop/Python Process Input Files/Input files old' ):
    fn=filename.split('.')[0]   
    pd.read_excel(filename).to_sql(fn, conn)

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'Region,S'

In [4]:
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in tables:
    print (name[0])

ADSA_CAfinance_EP
ADSA_Commutations_Loss
ADSA_Commutations_Prem
ADSA_Genius_Loss
ADSA_Genius_Prem
ADSA_IRISaviation_Claim
ADSA_IRISaviation_Prem
ADSA_IWEautoGL_Claim
ADSA_IWEautoGL_Prem
ADSA_IWEwc_Claim
ADSA_IWEwc_Prem
ADSA_Manuals_Loss
ADSA_Manuals_Prem
Company Mapping
Lookup_AcctCcyMap
Lookup_CCyFxRates
ResQ-HFM Mapping


In [5]:
SQL_UNION_ALL_LOSS= "SELECT ADSA_Genius_Loss.* FROM ADSA_Genius_Loss UNION ALL SELECT ADSA_Commutations_Loss.* FROM ADSA_Commutations_Loss UNION ALL SELECT ADSA_Manuals_Loss.* FROM ADSA_Manuals_Loss UNION ALL select ADSA_IWEautoGL_Claim.* from ADSA_IWEautoGL_Claim UNION ALL select adsa_IRISaviation_claim.* from adsa_IRISaviation_claim UNION ALL select ADSA_IWEwc_Claim.* from adsa_iwewc_claim "
SQL_UNION_ALL_PREMIUN = "SELECT ADSA_Genius_Prem.* FROM ADSA_Genius_Prem WHERE not ((ADSA_Genius_Prem.Source) = 'UBC' And (ADSA_Genius_Prem.Line = 'Property' )) UNION ALL  SELECT ADSA_Commutations_Prem.*  FROM ADSA_Commutations_Prem  UNION ALL SELECT ADSA_Manuals_Prem.*  FROM ADSA_Manuals_Prem  UNION ALL SELECT ADSA_IWEautoGL_Prem.*   FROM ADSA_IWEautoGL_Prem   UNION ALL SELECT ADSA_IWEwc_Prem.*  FROM ADSA_IWEwc_Prem  UNION ALL SELECT ADSA_CAfinance_EP.*   FROM ADSA_CAfinance_EP   WHERE (((ADSA_CAfinance_EP.Line) Is Not Null And (ADSA_CAfinance_EP.Line)<>'Asurion' And (ADSA_CAfinance_EP.Line)<>'Missing' And (ADSA_CAfinance_EP.Line)<>'Risk Sharing Pool'   and (ADSA_CAfinance_EP.line)<>'Kidnap & Ransom'))  UNION ALL SELECT ADSA_IRISaviation_prem.*  FROM ADSA_Irisaviation_prem  "

In [6]:
c=conn.cursor()
c.execute(SQL_UNION_ALL_LOSS)

UNION_ALL_LOSS_df = pd.read_sql_query(SQL_UNION_ALL_LOSS, conn)
UNION_ALL_PREMIUM_df = pd.read_sql_query(SQL_UNION_ALL_PREMIUN, conn)

In [7]:
import numpy as np
UNION_ALL_LOSS_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["LossQuarter", "AsOf"], how="any",inplace=True)
UNION_ALL_LOSS_df=UNION_ALL_LOSS_df[pd.notnull(UNION_ALL_LOSS_df['LossQuarter']) & pd.notnull(UNION_ALL_LOSS_df['AsOf'])]
UNION_ALL_LOSS_df.LossQuarter=UNION_ALL_LOSS_df.LossQuarter.astype('int64')
UNION_ALL_LOSS_df.AsOf=UNION_ALL_LOSS_df.AsOf.astype('int64')

UNION_ALL_PREMIUM_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["TransQuarter", "AsOf"], how="any",inplace=True)
UNION_ALL_PREMIUM_df=UNION_ALL_PREMIUM_df[pd.notnull(UNION_ALL_PREMIUM_df['TransQuarter']) & pd.notnull(UNION_ALL_PREMIUM_df['AsOf'])]
UNION_ALL_PREMIUM_df.TransQuarter=UNION_ALL_PREMIUM_df.TransQuarter.astype('int64')
UNION_ALL_PREMIUM_df.AsOf=UNION_ALL_PREMIUM_df.AsOf.astype('int64')

UNION_ALL_LOSS_df.to_sql("UNION_ALL_LOSS", conn)
UNION_ALL_PREMIUM_df.to_sql("UNION_ALL_PREMIUM", conn)

In [8]:
# c=conn.cursor()
# dropTableStatement = "DROP TABLE ResQ_WP"
# c.execute(dropTableStatement)

In [9]:
SQL_aResQ_Loss_Load_Step1=""" SELECT UNION_ALL_LOSS.Region, UNION_ALL_LOSS.Company, UNION_ALL_LOSS.Source, UNION_ALL_LOSS.Segment, UNION_ALL_LOSS.Line, UNION_ALL_LOSS.Subline1, UNION_ALL_LOSS.Subline2 
, case when UNION_ALL_LOSS.LossQuarter< ? then SUBSTR( ?,1,4)||'Q'||substr( ?,-1,1)  
else substr(UNION_ALL_LOSS.LossQuarter,1,4)||'Q'||substr(UNION_ALL_LOSS.LossQuarter,-1,1) end  LossQuarter

, case when (UNION_ALL_LOSS.LossQuarter< ? and UNION_ALL_LOSS.AsOf< ?) then 0   
when (UNION_ALL_LOSS.LossQuarter< ? and UNION_ALL_LOSS.AsOf>= ?) then (substr(UNION_ALL_LOSS.AsOf,1,4)*12+substr(UNION_ALL_LOSS.AsOf,-1,1)*3-substr( ?,1,4)*12-substr( ?,-1,1)*3) 
else (substr(UNION_ALL_LOSS.AsOf,1,4)*12+substr(UNION_ALL_LOSS.AsOf,-1,1)*3  -substr(UNION_ALL_LOSS.LossQuarter,1,4)*12-substr(UNION_ALL_LOSS.LossQuarter,-1,1)*3) end  DevPrd  

, UNION_ALL_LOSS.DataEntryType  

,sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_LOSS.Net_Inc_Loss+UNION_ALL_LOSS.Net_Inc_Rcvy)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius  
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_LOSS.Net_Inc_Loss+UNION_ALL_LOSS.Net_Inc_Rcvy)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM  
else (UNION_ALL_LOSS.Net_Inc_Loss+UNION_ALL_LOSS.Net_Inc_Rcvy)*Lookup_CcyFxRates.Rate_Genius end)  Net_Inc_Indemnity  

,sum(case when Lookup_AcctCcyMap.Country='AUS' then UNION_ALL_LOSS.Net_Inc_Exp*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius  
when Lookup_AcctCcyMap.Country='Canada' then  UNION_ALL_LOSS.Net_Inc_Exp*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM    
else UNION_ALL_LOSS.Net_Inc_Exp*Lookup_CcyFxRates.Rate_Genius end )  Net_Inc_ALAE      

,Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_LOSS.Net_Paid_Loss+UNION_ALL_LOSS.Net_Paid_Rcvy)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius   
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_LOSS.Net_Paid_Loss+ UNION_ALL_LOSS.Net_Paid_Rcvy)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM  
else (UNION_ALL_LOSS.Net_Paid_Loss+UNION_ALL_LOSS.Net_Paid_Rcvy)*Lookup_CcyFxRates.Rate_Genius end )  Net_Paid_Indemnity   

,Sum(case when Lookup_AcctCcyMap.Country='AUS' then UNION_ALL_LOSS.Net_Paid_Exp*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius   
when Lookup_AcctCcyMap.Country='Canada' then UNION_ALL_LOSS.Net_Paid_Exp*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM   
else UNION_ALL_LOSS.Net_Paid_Exp*Lookup_CcyFxRates.Rate_Genius end)   \
 Net_Paid_ALAE  \

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_LOSS.CededP_Inc_Loss+UNION_ALL_LOSS.CededP_Inc_Rcvy+UNION_ALL_LOSS.CededNP_Inc_Loss+UNION_ALL_LOSS.CedednP_Inc_Rcvy)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius  \
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_LOSS.Cededp_Inc_Loss+UNION_ALL_LOSS.Cededp_Inc_Rcvy+UNION_ALL_LOSS.Cedednp_Inc_Loss+UNION_ALL_LOSS.Cedednp_Inc_Rcvy)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM  \
else (UNION_ALL_LOSS.Cededp_Inc_Loss+UNION_ALL_LOSS.Cededp_Inc_Rcvy+UNION_ALL_LOSS.Cedednp_Inc_Loss+UNION_ALL_LOSS.Cedednp_Inc_Rcvy)*Lookup_CcyFxRates.Rate_Genius end)  
 Ceded_Inc_Indemnity  

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_LOSS.Cededp_Inc_Exp+UNION_ALL_LOSS.Cedednp_Inc_Exp)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius   
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_LOSS.Cededp_Inc_Exp+UNION_ALL_LOSS.Cedednp_Inc_Exp)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM    
else (UNION_ALL_LOSS.Cededp_Inc_Exp+UNION_ALL_LOSS.Cedednp_Inc_Exp)*Lookup_CcyFxRates.Rate_Genius end)    
 Ceded_Inc_ALAE  

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_LOSS.Cededp_Paid_Loss+UNION_ALL_LOSS.Cededp_Paid_Rcvy+UNION_ALL_LOSS.Cedednp_Paid_Loss+UNION_ALL_LOSS.Cedednp_Paid_Rcvy)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius   \
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_LOSS.Cededp_Paid_Loss+UNION_ALL_LOSS.Cededp_Paid_Rcvy+UNION_ALL_LOSS.Cedednp_Paid_Loss+UNION_ALL_LOSS.Cedednp_Paid_Rcvy)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM   \
else (UNION_ALL_LOSS.Cededp_Paid_Loss+UNION_ALL_LOSS.Cededp_Paid_Rcvy+UNION_ALL_LOSS.Cedednp_Paid_Loss+UNION_ALL_LOSS.Cedednp_Paid_Rcvy)*Lookup_CcyFxRates.Rate_Genius end)  Ceded_Paid_Indemnity  

, Sum(case when Lookup_AcctCcyMap.Country='AUS'then (UNION_ALL_LOSS.Cededp_Paid_Exp+UNION_ALL_LOSS.Cedednp_Paid_Exp)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius  
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_LOSS.Cededp_Paid_Exp+UNION_ALL_LOSS.Cedednp_Paid_Exp)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM    
else (UNION_ALL_LOSS.Cededp_Paid_Exp+UNION_ALL_LOSS.Cedednp_Paid_Exp)*Lookup_CcyFxRates.Rate_Genius end)   
 Ceded_Paid_ALAE   

FROM 
(UNION_ALL_LOSS LEFT JOIN (Lookup_AcctCcyMap LEFT JOIN Lookup_CcyFxRates AS Lookup_CcyFxRates_1 ON Lookup_AcctCcyMap.AcctCcy = Lookup_CcyFxRates_1.Currency) ON UNION_ALL_LOSS.Source = Lookup_AcctCcyMap.CountryMap) LEFT JOIN Lookup_CcyFxRates ON UNION_ALL_LOSS.Original = Lookup_CcyFxRates.Currency   \
GROUP BY  

 UNION_ALL_LOSS.Region   
 , UNION_ALL_LOSS.Company   
 , UNION_ALL_LOSS.Source   
 , UNION_ALL_LOSS.Segment   
 , UNION_ALL_LOSS.Line   
 , UNION_ALL_LOSS.Subline1   
 , UNION_ALL_LOSS.Subline2   
 , case when UNION_ALL_LOSS.LossQuarter< ? then SUBSTR( ?,1,4)||'Q'||substr( ?,-1,1) else substr(UNION_ALL_LOSS.LossQuarter,1,4)||'Q'||substr(UNION_ALL_LOSS.LossQuarter,-1,1) end    

, case when (UNION_ALL_LOSS.LossQuarter< ? and UNION_ALL_LOSS.AsOf< ?) then 0   \
when (UNION_ALL_LOSS.LossQuarter< ? and UNION_ALL_LOSS.AsOf>= ?) then (substr(UNION_ALL_LOSS.AsOf,1,4)*12+substr(UNION_ALL_LOSS.AsOf,-1,1)*3-substr( ?,1,4)*12-substr( ?,-1,1)*3) 
else (substr(UNION_ALL_LOSS.AsOf,1,4)*12+substr(UNION_ALL_LOSS.AsOf,-1,1)*3 -substr(UNION_ALL_LOSS.LossQuarter,1,4)*12-substr(UNION_ALL_LOSS.LossQuarter,-1,1)*3) end

, UNION_ALL_LOSS.DataEntryType"""

In [10]:
aResQ_Loss_Load_Step1_df = pd.read_sql_query(SQL_aResQ_Loss_Load_Step1, conn,params=parameter_loss)
aResQ_Loss_Load_Step1_df['LossQuarter_ts']=pd.to_datetime(aResQ_Loss_Load_Step1_df.LossQuarter)
aResQ_Loss_Load_Step1_df['CalendarQuarter']=np.NaN

def add_DevPrd(row):
    return row['LossQuarter_ts']+pd.DateOffset(months = row['DevPrd'])

aResQ_Loss_Load_Step1_df['CQ']=aResQ_Loss_Load_Step1_df.apply(add_DevPrd, axis=1)

aResQ_Loss_Load_Step1_df['CQ_y']=aResQ_Loss_Load_Step1_df['CQ'].dt.year
aResQ_Loss_Load_Step1_df['CQ_q']=aResQ_Loss_Load_Step1_df['CQ'].dt.quarter

def CalendarQuarter(row):
    return str(row['CQ_y'])+'Q'+ str(row['CQ_q'])

aResQ_Loss_Load_Step1_df['CalendarQuarter']=aResQ_Loss_Load_Step1_df.apply(CalendarQuarter, axis=1)
aResQ_Loss_Load_Step1_df.to_sql("aResQ_Loss_Load", conn)

In [11]:
SQL_aResQ_Prem_Load_Step1="""

SELECT 
UNION_ALL_PREMIUM.Region
, UNION_ALL_PREMIUM.Company
, UNION_ALL_PREMIUM.Source
, UNION_ALL_PREMIUM.Segment
, UNION_ALL_PREMIUM.Line
, UNION_ALL_PREMIUM.Subline1
, UNION_ALL_PREMIUM.Subline2

, case when UNION_ALL_PREMIUM.TransQuarter<? then substr(?,1,4)||'Q'|| substr(?,-1,1) else substr(UNION_ALL_PREMIUM.TransQuarter,1,4)||'Q' || substr(UNION_ALL_PREMIUM.TransQuarter,-1,1) end TransQuarter2

, case when UNION_ALL_PREMIUM.AsOf<? then substr(?,1,4)||'Q'|| substr(?,-1,1) else substr(UNION_ALL_PREMIUM.AsOf,1,4) || 'Q' || substr(UNION_ALL_PREMIUM.AsOf,-1,1) end CalendarQuarter

, UNION_ALL_PREMIUM.DataEntryType

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then UNION_ALL_PREMIUM.Net_WP_PQ*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius
when Lookup_AcctCcyMap.Country='Canada' then UNION_ALL_PREMIUM.Net_WP_PQ*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else UNION_ALL_PREMIUM.Net_WP_PQ*Lookup_CcyFxRates.Rate_Genius end)  Net_WP_PQ
, Sum(case when Lookup_AcctCcyMap.Country='AUS' then UNION_ALL_PREMIUM.Net_WP_TQ*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius
when Lookup_AcctCcyMap.Country='Canada' then UNION_ALL_PREMIUM.Net_WP_TQ*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else UNION_ALL_PREMIUM.Net_WP_TQ*Lookup_CcyFxRates.Rate_Genius end )  Net_WP_TQ
, Sum(case when Lookup_AcctCcyMap.Country='AUS' then UNION_ALL_PREMIUM.Net_EP*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then UNION_ALL_PREMIUM.Net_EP*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else UNION_ALL_PREMIUM.Net_EP*Lookup_CcyFxRates.Rate_Genius end )  Net_EP
, Sum(case when Lookup_AcctCcyMap.Country='AUS' then UNION_ALL_PREMIUM.Net_Unx_EP*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then UNION_ALL_PREMIUM.Net_Unx_EP*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else UNION_ALL_PREMIUM.Net_Unx_EP*Lookup_CcyFxRates.Rate_Genius end )  Net_Unx_EP
, Sum(case when Lookup_AcctCcyMap.Country='AUS' then UNION_ALL_PREMIUM.Net_Unx_Inc*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then UNION_ALL_PREMIUM.Net_Unx_Inc*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else UNION_ALL_PREMIUM.Net_Unx_Inc*Lookup_CcyFxRates.Rate_Genius end)  Net_Unx_Inc

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_PREMIUM.Ceded_WP_PQ_nonProp+UNION_ALL_PREMIUM.Ceded_WP_PQ_Prop)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_PREMIUM.Ceded_WP_PQ_nonProp+UNION_ALL_PREMIUM.Ceded_WP_PQ_Prop)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else (UNION_ALL_PREMIUM.Ceded_WP_PQ_nonProp+UNION_ALL_PREMIUM.Ceded_WP_PQ_Prop)*Lookup_CcyFxRates.Rate_Genius end )  Ceded_WP_PQ

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_PREMIUM.Ceded_WP_TQ_prop+UNION_ALL_PREMIUM.Ceded_WP_TQ_nonprop)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_PREMIUM.Ceded_WP_TQ_prop+UNION_ALL_PREMIUM.Ceded_WP_TQ_nonprop)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else (UNION_ALL_PREMIUM.Ceded_WP_TQ_prop+UNION_ALL_PREMIUM.Ceded_WP_TQ_nonprop)*Lookup_CcyFxRates.Rate_Genius end)  Ceded_WP_TQ

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_PREMIUM.Ceded_EP_Prop+UNION_ALL_PREMIUM.Ceded_EP_nonProp)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_PREMIUM.Ceded_EP_Prop+UNION_ALL_PREMIUM.Ceded_EP_nonProp)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else (UNION_ALL_PREMIUM.Ceded_EP_Prop+UNION_ALL_PREMIUM.Ceded_EP_nonProp)*Lookup_CcyFxRates.Rate_Genius end )  Ceded_EP

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_PREMIUM.Ceded_Unx_EP_Prop+UNION_ALL_PREMIUM.Ceded_Unx_EP_nonProp)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_PREMIUM.Ceded_Unx_EP_Prop+UNION_ALL_PREMIUM.Ceded_Unx_EP_nonProp)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else (UNION_ALL_PREMIUM.Ceded_Unx_EP_Prop+UNION_ALL_PREMIUM.Ceded_Unx_EP_nonProp)*Lookup_CcyFxRates.Rate_Genius end )  Ceded_Unx_EP

, Sum(case when Lookup_AcctCcyMap.Country='AUS' then (UNION_ALL_PREMIUM.Ceded_Unx_Inc_Prop+UNION_ALL_PREMIUM.Ceded_Unx_Inc_nonProp)*Lookup_CcyFxRates.Rate_Genius/Lookup_CcyFxRates_1.Rate_Genius 
when Lookup_AcctCcyMap.Country='Canada' then (UNION_ALL_PREMIUM.Ceded_Unx_Inc_Prop+UNION_ALL_PREMIUM.Ceded_Unx_Inc_nonProp)*Lookup_CcyFxRates.Rate_HFM/Lookup_CcyFxRates_1.Rate_HFM
else (UNION_ALL_PREMIUM.Ceded_Unx_Inc_Prop+UNION_ALL_PREMIUM.Ceded_Unx_Inc_nonProp)*Lookup_CcyFxRates.Rate_Genius end )  Ceded_Unx_Inc

FROM 
((UNION_ALL_PREMIUM LEFT JOIN Lookup_AcctCcyMap ON UNION_ALL_PREMIUM.Source = Lookup_AcctCcyMap.CountryMap) 
LEFT JOIN Lookup_CcyFxRates ON UNION_ALL_PREMIUM.OriginalCcy = Lookup_CcyFxRates.Currency) LEFT JOIN Lookup_CcyFxRates  Lookup_CcyFxRates_1 ON Lookup_AcctCcyMap.AcctCcy = Lookup_CcyFxRates_1.Currency

GROUP BY

UNION_ALL_PREMIUM.Region
, UNION_ALL_PREMIUM.Company
, UNION_ALL_PREMIUM.Source
, UNION_ALL_PREMIUM.Segment
, UNION_ALL_PREMIUM.Line
, UNION_ALL_PREMIUM.Subline1
, UNION_ALL_PREMIUM.Subline2

, case when UNION_ALL_PREMIUM.TransQuarter< ? then substr(?,1,4)||'Q'|| substr(?,-1,1) else substr(UNION_ALL_PREMIUM.TransQuarter,1,4)||'Q' || substr(UNION_ALL_PREMIUM.TransQuarter,-1,1) end

, case when UNION_ALL_PREMIUM.AsOf< ? then substr(?,1,4)||'Q'|| substr(?,-1,1) else substr(UNION_ALL_PREMIUM.AsOf,1,4) || 'Q' || substr(UNION_ALL_PREMIUM.AsOf,-1,1) end 

, UNION_ALL_PREMIUM.DataEntryType

"""

In [12]:
c.execute(SQL_aResQ_Loss_Load_Step1,(PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter,PriorQuarter))

In [13]:
aResQ_Prem_Load_Step1_df = pd.read_sql_query(SQL_aResQ_Prem_Load_Step1, conn,params=parameter_prem)
aResQ_Prem_Load_Step1_df.to_sql("aResQ_Prem_Load_Step1", conn)

In [14]:
SQL_aResQ_Prem_Load_Step3 ="""

SELECT 
aResQ_Prem_Load_Step1.Region AS Region
, aResQ_Prem_Load_Step1.Company AS Company
, aResQ_Prem_Load_Step1.Source AS Source
, aResQ_Prem_Load_Step1.Segment AS Segment
, aResQ_Prem_Load_Step1.Line AS Line
, aResQ_Prem_Load_Step1.SubLine1 AS SubLine1
, aResQ_Prem_Load_Step1.SubLine2 AS SubLine2
, aResQ_Prem_Load_Step1.TransQuarter2 AS TransQuarter
, aResQ_Prem_Load_Step1.CalendarQuarter AS CalendarQuarter
, aResQ_Prem_Load_Step1.DataEntryType AS DataEntryType
, Sum(aResQ_Prem_Load_Step1.Net_WP_PQ) AS SumOfNet_WP_PQ
, Sum(aResQ_Prem_Load_Step1.Net_WP_TQ) AS SumOfNet_WP_TQ
, Sum(aResQ_Prem_Load_Step1.Net_EP) AS SumOfNet_EP
, Sum(aResQ_Prem_Load_Step1.Net_Unx_EP) AS SumOfNet_Unx_EP
, Sum(aResQ_Prem_Load_Step1.Net_Unx_Inc) AS SumOfNet_Unx_Inc
, Sum(aResQ_Prem_Load_Step1.Ceded_WP_PQ) AS SumOfCeded_WP_PQ
, Sum(aResQ_Prem_Load_Step1.Ceded_WP_TQ) AS SumOfCeded_WP_TQ
, Sum(aResQ_Prem_Load_Step1.Ceded_EP) AS SumOfCeded_EP
, Sum(aResQ_Prem_Load_Step1.Ceded_Unx_EP) AS SumOfCeded_Unx_EP
, Sum(aResQ_Prem_Load_Step1.Ceded_Unx_Inc) AS SumOfCeded_Unx_Inc 
FROM aResQ_Prem_Load_Step1
GROUP BY 
aResQ_Prem_Load_Step1.Region
, aResQ_Prem_Load_Step1.Company
, aResQ_Prem_Load_Step1.Source
, aResQ_Prem_Load_Step1.Segment
, aResQ_Prem_Load_Step1.Line
, aResQ_Prem_Load_Step1.SubLine1
, aResQ_Prem_Load_Step1.SubLine2
, aResQ_Prem_Load_Step1.TransQuarter2
, aResQ_Prem_Load_Step1.CalendarQuarter
, aResQ_Prem_Load_Step1.DataEntryType

"""

aResQ_Prem_Load_Step3_df = pd.read_sql_query(SQL_aResQ_Prem_Load_Step3, conn)

aResQ_Prem_Load_Step3_df.to_sql("aResQ_Prem_Load", conn)

In [15]:
####### CHANGE AsAtPeriod to current YYYYMM 

SQL_Premiums="""

SELECT 
aResQ_Prem_Load.Region
, aResQ_Prem_Load.Company
, aResQ_Prem_Load.Source
, aResQ_Prem_Load.Segment
, aResQ_Prem_Load.Line
, aResQ_Prem_Load.SubLine1
, aResQ_Prem_Load.SubLine2
, aResQ_Prem_Load.TransQuarter
, aResQ_Prem_Load.CalendarQuarter
, aResQ_Prem_Load.DataEntryType
, aResQ_Prem_Load.sumofNet_wp_Pq AS Net_WP
, aResQ_Prem_Load.sumofceded_wp_pq AS Ceded_WP
, aResQ_Prem_Load.SumOfNet_EP AS Net_EP
, aResQ_Prem_Load.SumOfCeded_EP AS Ceded_EP
, ? AS AsAtPeriod
FROM aResQ_Prem_Load;

"""

In [16]:
Premiums_df = pd.read_sql_query(SQL_Premiums, conn,params=(AsAtPeriod,))

Premiums_df.to_sql("Premiums", conn)

In [17]:
company_mapping_df=pd.read_excel('Company Mapping.xlsx')
ResQ_HFM_Mapping_df=pd.read_excel('ResQ-HFM Mapping.xlsx')
temp1=Premiums_df.loc[Premiums_df['CalendarQuarter']==current_qtr,:]
temp2=pd.melt(temp1,id_vars=['Region', 'Company','Source','Segment','Line','SubLine1','SubLine2','TransQuarter','CalendarQuarter','DataEntryType','AsAtPeriod'],var_name=['Prem_type'], value_name='value')
temp2['Reserve Class']=temp2[['Region', 'Source','Segment','Line','SubLine1','SubLine2']].apply(lambda x: '\\'.join(x.dropna()), axis=1)
temp3=temp2.merge(company_mapping_df, on= 'Company')
temp4=temp3.merge(ResQ_HFM_Mapping_df, on='Reserve Class')
temp4['Region']=temp4['Region'].apply(lambda x: x.split('-')[1])


In [18]:
temp4.loc[(temp4['Source']=='AU') & (temp4['HFM']=='Crisis Management'),:].groupby('Prem_type').agg('sum')

,AsAtPeriod,value
Prem_type,,
Ceded_EP,4441932,1.016796e+06
Ceded_WP,4441932,1.416616e+06
Net_EP,4441932,1.919676e+06
Net_WP,4441932,2.386730e+06


In [19]:
SQL_Claims="""

SELECT 
aResQ_Loss_Load.Region
, aResQ_Loss_Load.Company
, aResQ_Loss_Load.Source
, aResQ_Loss_Load.Segment
, aResQ_Loss_Load.Line
, aResQ_Loss_Load.Subline1
, aResQ_Loss_Load.Subline2
, aResQ_Loss_Load.LossQuarter
, aResQ_Loss_Load.DevPrd

, aResQ_Loss_Load.CalendarQuarter

, aResQ_Loss_Load.DataEntryType
, Sum(Net_INC_Indemnity+Net_Inc_ALAE) AS Net_Inc
, Sum(Net_paid_indemnity+net_paid_alae) AS Net_Paid
, Sum(ceded_inc_indemnity+ceded_inc_alae) AS Ceded_Inc
, Sum(ceded_paid_indemnity+ceded_paid_alae) AS Ceded_Paid
, ? AS AsAtPeriod
FROM aResQ_Loss_Load
GROUP BY 
aResQ_Loss_Load.Region
, aResQ_Loss_Load.Company
, aResQ_Loss_Load.Source
, aResQ_Loss_Load.Segment
, aResQ_Loss_Load.Line
, aResQ_Loss_Load.Subline1
, aResQ_Loss_Load.Subline2
, aResQ_Loss_Load.LossQuarter
, aResQ_Loss_Load.DevPrd
, aResQ_Loss_Load.CalendarQuarter
, aResQ_Loss_Load.DataEntryType, ?


"""
Claims_df = pd.read_sql_query(SQL_Claims, conn,params=(AsAtPeriod,AsAtPeriod,))
Claims_df.to_sql("Claims", conn)

In [20]:
Claims_df.rename(columns={'Region': 'Region_old', 'Source': 'Source_old'}, inplace=True)
Premiums_df.rename(columns={'Region': 'Region_old', 'Source': 'Source_old'}, inplace=True)

def region(row):
    if row['Region_old']=='LIU-US' and row['Segment']=='First Party' and row['Line']!='E&S Property':
        return 'GS-MEC'
    elif row['Segment']=='Other Commercial Specialty' and row['Line']=='Terrorism':
        return 'GS-MEC'
    else:
        return row['Region_old']

def source(row):
    if row['Region_old']=='LIU-US' and row['Segment']=='First Party' and row['Line']!='E&S Property':
        return 'MEC'
    elif row['Segment']=='Other Commercial Specialty' and row['Line']=='Terrorism':
        return 'MEC'
    else:
        return row['Source_old']
    
import math 
def fixed_cal_qtr(row):
    qtr_para1=(int(row['LossQuarter'][-1])+row['DevPrd']/3)/4- math.floor((int(row['LossQuarter'][-1])+row['DevPrd']/3)/4)
    if qtr_para1*4==0:
        qtr_para2=4
    else:
        qtr_para2=qtr_para1*4
    return str(int(row['LossQuarter'][:4])+  int((int(row['LossQuarter'][-1])*3+ row['DevPrd']-1)/12))+'Q'+ str(int(qtr_para2))

    
Claims_df['Region']=Claims_df.apply(region, axis=1)
Claims_df['Source']=Claims_df.apply(source, axis=1)
Claims_df['Fixed Calendar Quarter']=Claims_df.apply(fixed_cal_qtr, axis=1)
Premiums_df['Region']=Premiums_df.apply(region, axis=1)
Premiums_df['Source']=Premiums_df.apply(source, axis=1)


Claims_df.to_sql("ResQ_Claims", conn)
Premiums_df.to_sql("ResQ_WP", conn)



C:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [21]:
Premiums_df.loc[Premiums_df['Region']=='GS-MEC',:]

,Region_old,Company,Source_old,Segment,Line,SubLine1,SubLine2,TransQuarter,CalendarQuarter,DataEntryType,Net_WP,Ceded_WP,Net_EP,Ceded_EP,AsAtPeriod,Region,Source
243977,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2016Q1,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243978,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2016Q2,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243979,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2016Q3,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243980,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2016Q4,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243981,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2017Q1,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243982,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2017Q2,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243983,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2017Q3,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243984,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2017Q4,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243985,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2018Q1,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC
243986,LIU-LA,BRASIL,LA,Other Commercial Specialty,Terrorism,None,None,2008Q1,2018Q2,Standard,0.000000,0.000000,0.000000,0.000000,201906,GS-MEC,MEC


In [22]:
SQL_EP1="""
SELECT 
substr(ResQ_WP.Region,-2,2) AS Region_
, [Company Mapping].Entity
, ResQ_WP.Company
, case when ResQ_WP.subline1 is null then ResQ_WP.Region || '\\' || ResQ_WP.Source || '\\' || ResQ_WP.Segment || '\\' || ResQ_WP.Line
    when ResQ_WP.subline1 is not null and ResQ_WP.subline2 is null then ResQ_WP.Region || '\\' || ResQ_WP.Source || '\\' || ResQ_WP.Segment || '\\' || ResQ_WP.Line || '\\' || ResQ_WP.subline1
    else ResQ_WP.Region || '\\' || ResQ_WP.Source || '\\' || ResQ_WP.Segment || '\\' || ResQ_WP.Line || '\\' || ResQ_WP.subline1 || '\\' || ResQ_WP.subline2 end AS Reserve_Class1
, ResQ_WP.CalendarQuarter AS CQ
, ResQ_WP.DataEntryType
, ResQ_WP.Net_EP
, ResQ_WP.Ceded_EP
, ResQ_WP.AsAtPeriod
FROM 
ResQ_WP 
LEFT JOIN 
[Company Mapping] ON ResQ_WP.Company = [Company Mapping].Company
WHERE ResQ_WP.Region!='Missing'

"""
SQL_C1="""
SELECT 
substr(ResQ_Claims.Region,-2,2) AS Region_
, [Company Mapping].Entity
, ResQ_Claims.Company
, case when ResQ_Claims.subline1 is null then ResQ_Claims.Region || '\\' || ResQ_Claims.Source || '\\' || ResQ_Claims.Segment || '\\' || ResQ_Claims.Line
    when ResQ_Claims.subline1 is not null and ResQ_Claims.subline2 is null then ResQ_Claims.Region || '\\' || ResQ_Claims.Source || '\\' || ResQ_Claims.Segment || '\\' || ResQ_Claims.Line || '\\' || ResQ_Claims.subline1
    else ResQ_Claims.Region || '\\' || ResQ_Claims.Source || '\\' || ResQ_Claims.Segment || '\\' || ResQ_Claims.Line || '\\' || ResQ_Claims.subline1 || '\\' || ResQ_Claims.subline2 end AS Reserve_Class1
, ResQ_Claims.CalendarQuarter AS CQ
, ResQ_Claims.DataEntryType
, ResQ_Claims.Net_Inc
, ResQ_Claims.Net_Paid
, ResQ_Claims.Ceded_Inc
, ResQ_Claims.Ceded_Paid
, ResQ_Claims.AsAtPeriod
FROM 
ResQ_Claims 
LEFT JOIN 
[Company Mapping] ON ResQ_Claims.Company = [Company Mapping].Company
"""

SQL_WP1="""
SELECT 
substr(ResQ_WP.Region,-2,2) AS Region_
, [Company Mapping].Entity
, ResQ_WP.Company
, case when ResQ_WP.subline1 is null then ResQ_WP.Region || '\\' || ResQ_WP.Source || '\\' || ResQ_WP.Segment || '\\' || ResQ_WP.Line
    when ResQ_WP.subline1 is not null and ResQ_WP.subline2 is null then ResQ_WP.Region || '\\' || ResQ_WP.Source || '\\' || ResQ_WP.Segment || '\\' || ResQ_WP.Line || '\\' || ResQ_WP.subline1
    else ResQ_WP.Region || '\\' || ResQ_WP.Source || '\\' || ResQ_WP.Segment || '\\' || ResQ_WP.Line || '\\' || ResQ_WP.subline1 || '\\' || ResQ_WP.subline2 end AS Reserve_Class1
, ResQ_WP.CalendarQuarter AS CQ
, ResQ_WP.DataEntryType
, ResQ_WP.Net_WP
, ResQ_WP.Ceded_WP
, ResQ_WP.AsAtPeriod

FROM ResQ_WP LEFT JOIN 
[Company Mapping] ON ResQ_WP.Company = [Company Mapping].Company

WHERE ResQ_WP.Region!='Missing'

"""

EP1_df = pd.read_sql_query(SQL_EP1, conn)
C1_df=pd.read_sql_query(SQL_C1, conn)
WP1_df=pd.read_sql_query(SQL_WP1, conn)

EP1_df.to_sql("EP1", conn)
C1_df.to_sql("C1", conn)
WP1_df.to_sql("WP1", conn)

In [23]:
SQL_WP2_ceded_wp="""

SELECT 
WP1.Region_
, WP1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
, "Ceded WP" AS Measure
, WP1.CQ
, WP1.DataEntryType
, WP1.Ceded_WP
, WP1.Net_WP

FROM WP1 
LEFT JOIN [ResQ-HFM Mapping] ON WP1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE WP1.Entity!='CA-UBC'
and WP1.Entity!='CA-CBO'
and WP1.Entity!='CA-CBU'

"""
WP2_ceded_wp_df = pd.read_sql_query(SQL_WP2_ceded_wp, conn)
WP2_ceded_wp_df=WP2_ceded_wp_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Ceded_WP',aggfunc='sum').reset_index()
WP2_ceded_wp_df.fillna(0, inplace=True)
WP2_ceded_wp_df['Recast']=WP2_ceded_wp_df['Recast_LOD']+WP2_ceded_wp_df['Recast_RAD']
WP2_ceded_wp_df.rename(columns={'Bulk_Ceded': 'Bulk_Entries'}, inplace=True)

WP2_ceded_wp_df['Other'] = pd.DataFrame([np.nan])
WP2_ceded_wp_df.drop(['Recast_LOD','Recast_RAD'],axis=1,inplace=True)

SQL_WP2_gross_wp="""

SELECT 
WP1.Region_
, WP1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
, "DIRECT&ASSUMED WP" AS Measure
, WP1.CQ
, WP1.DataEntryType
, WP1.Ceded_WP
, WP1.Net_WP

FROM WP1 
LEFT JOIN [ResQ-HFM Mapping] ON WP1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE WP1.Entity!='CA-UBC'
and WP1.Entity!='CA-CBO'
and WP1.Entity!='CA-CBU'

"""
WP2_gross_wp_df = pd.read_sql_query(SQL_WP2_gross_wp, conn)
WP2_gross_wp_df.fillna(0, inplace=True)
WP2_gross_wp_df['Gross_WP']=WP2_gross_wp_df['Ceded_WP']+WP2_gross_wp_df['Net_WP']
WP2_gross_wp_df=WP2_gross_wp_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Gross_WP',aggfunc='sum').reset_index()
WP2_gross_wp_df.fillna(0, inplace=True)
WP2_gross_wp_df['Recast']=WP2_gross_wp_df['Recast_LOD']+WP2_gross_wp_df['Recast_RAD']
WP2_gross_wp_df.rename(columns={'Bulk_Ceded': 'Bulk_Entries'}, inplace=True)
WP2_gross_wp_df['Other'] = pd.DataFrame([np.nan])
WP2_gross_wp_df.drop(['Recast_LOD','Recast_RAD'],axis=1,inplace=True)

SQL_EP2_ceded_ep="""

SELECT 
EP1.Region_
, EP1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
, "Ceded EP" AS Measure
, EP1.CQ
, EP1.DataEntryType
, EP1.Ceded_EP
, EP1.Net_EP

FROM EP1 
LEFT JOIN [ResQ-HFM Mapping] ON EP1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE EP1.Entity!='CA-UBC'
and EP1.Entity!='CA-CBO'
and EP1.Entity!='CA-CBU'

"""
EP2_ceded_ep_df = pd.read_sql_query(SQL_EP2_ceded_ep, conn)
EP2_ceded_ep_df=EP2_ceded_ep_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Ceded_EP',aggfunc='sum').reset_index()

EP2_ceded_ep_df.fillna(0, inplace=True)
EP2_ceded_ep_df['Recast']=EP2_ceded_ep_df['Recast_LOD']+EP2_ceded_ep_df['Recast_RAD']
EP2_ceded_ep_df.rename(columns={'Bulk_Ceded': 'Bulk_Entries'}, inplace=True)

EP2_ceded_ep_df['Other'] = pd.DataFrame([np.nan])
EP2_ceded_ep_df.drop(['Recast_LOD','Recast_RAD'],axis=1,inplace=True)


SQL_EP2_gross_ep="""

SELECT 
EP1.Region_
, EP1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
, "DIRECT&ASSUMED EP" AS Measure
, EP1.CQ
, EP1.DataEntryType
, EP1.Ceded_EP
, EP1.Net_EP

FROM EP1 
LEFT JOIN [ResQ-HFM Mapping] ON EP1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE EP1.Entity!='CA-UBC'
and EP1.Entity!='CA-CBO'
and EP1.Entity!='CA-CBU'

"""
EP2_gross_ep_df = pd.read_sql_query(SQL_EP2_gross_ep, conn)
EP2_gross_ep_df.fillna(0, inplace=True)
EP2_gross_ep_df['Gross_EP']=EP2_gross_ep_df['Ceded_EP']+EP2_gross_ep_df['Net_EP']
EP2_gross_ep_df=EP2_gross_ep_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Gross_EP',aggfunc='sum').reset_index()

EP2_gross_ep_df.fillna(0, inplace=True)
EP2_gross_ep_df['Recast']=EP2_gross_ep_df['Recast_LOD']+EP2_gross_ep_df['Recast_RAD']
EP2_gross_ep_df.rename(columns={'Bulk_Ceded': 'Bulk_Entries'}, inplace=True)
EP2_gross_ep_df['Other'] = pd.DataFrame([np.nan])
EP2_gross_ep_df.drop(['Recast_LOD','Recast_RAD'],axis=1,inplace=True)

In [24]:
SQL_C2_ceded_os="""

SELECT 
C1.Region_
, C1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
, "CHANGE IN CEDED CASE LOSS&ALAE" AS Measure
, C1.CQ
,C1.DataEntryType
,C1.Ceded_Inc
,C1.Ceded_Paid


FROM C1 
LEFT JOIN [ResQ-HFM Mapping] ON C1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE C1.Entity!='CA-UBC'
and C1.Entity!='ca-cbo'
and C1.Entity!='ca-cbu'

"""
C2_ceded_os_df = pd.read_sql_query(SQL_C2_ceded_os, conn)
C2_ceded_os_df.fillna(0,inplace=True)
C2_ceded_os_df['Ceded_OS']=C2_ceded_os_df['Ceded_Inc']-C2_ceded_os_df['Ceded_Paid']
C2_ceded_os_df=C2_ceded_os_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Ceded_OS',aggfunc='sum').reset_index()

SQL_C2_ceded_paid="""
SELECT 
C1.Region_
, C1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
, "CEDED LOSS&ALAE PAID" AS Measure
, C1.CQ
,C1.DataEntryType
,C1.Ceded_Paid
FROM C1 
LEFT JOIN [ResQ-HFM Mapping] ON C1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE C1.Entity!='CA-UBC'


"""
C2_ceded_paid_df = pd.read_sql_query(SQL_C2_ceded_paid, conn)
C2_ceded_paid_df=C2_ceded_paid_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Ceded_Paid',aggfunc='sum').reset_index()



SQL_C2_gross_paid="""
SELECT 
C1.Region_
, C1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
,  "Direct&Assumed Loss&ALAE Paid"  AS Measure
, C1.CQ
, C1.DataEntryType
, C1.Ceded_Paid
, C1.Net_Paid
FROM C1 
LEFT JOIN [ResQ-HFM Mapping] ON C1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE C1.Entity!='CA-UBC'

"""
C2_gross_paid_df = pd.read_sql_query(SQL_C2_gross_paid, conn)
C2_gross_paid_df.fillna(0, inplace=True)
C2_gross_paid_df['Gross_Paid']=C2_gross_paid_df['Net_Paid']+C2_gross_paid_df['Ceded_Paid']
C2_gross_paid_df=C2_gross_paid_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Gross_Paid',aggfunc='sum').reset_index()


SQL_C2_gross_os="""

SELECT 
C1.Region_
, C1.Entity
, [ResQ-HFM Mapping].HFM
, [ResQ-HFM Mapping].ResQ AS Reserve_Class
, "CHANGE IN DIRECT&ASSUMED CASE LOSS&ALAE" AS Measure
, C1.CQ
, C1.DataEntryType
, C1.Net_Inc
, C1.Ceded_Inc
, C1.Ceded_Paid
, C1.Net_Paid

FROM C1 
LEFT JOIN [ResQ-HFM Mapping] ON C1.[Reserve_Class1] = [ResQ-HFM Mapping].[Reserve Class]

WHERE C1.Entity!='CA-UBC'

"""
C2_gross_os_df = pd.read_sql_query(SQL_C2_gross_os, conn)
C2_gross_os_df.fillna(0, inplace=True)
C2_gross_os_df['Gross_OS']=C2_gross_os_df['Net_Inc']+C2_gross_os_df['Ceded_Inc']-C2_gross_os_df['Net_Paid']-C2_gross_os_df['Ceded_Paid']
C2_gross_os_df=C2_gross_os_df.pivot_table(index=['Region_','Entity','HFM','Reserve_Class','Measure','CQ'],columns='DataEntryType',values='Gross_OS',aggfunc='sum').reset_index()


In [25]:
P3_columns=['Region_','Entity','HFM','Reserve_Class','Measure','CQ','Standard','Manual','Recast','Bulk_Entries','Other']
P3_temp_df=pd.concat([EP2_gross_ep_df[P3_columns],
                      EP2_ceded_ep_df[P3_columns],
                      WP2_gross_wp_df[P3_columns],
                      WP2_ceded_wp_df[P3_columns]], ignore_index=True)
P3_temp_df.rename(columns={'Manual': 'Manuals'}, inplace=True)

C3_columns=['Region_','Entity','HFM','Reserve_Class','Measure','CQ','Standard','Manuals']   
C3_temp_df=pd.concat([C2_gross_os_df[C3_columns],
                    C2_gross_paid_df[C3_columns],
                    C2_ceded_paid_df[C3_columns],
                    C2_ceded_os_df[C3_columns]], ignore_index=True)

C3_temp_df[['Recast','Bulk_Entries','Other']] = pd.DataFrame([[np.nan, np.nan, np.nan]], index=C3_temp_df.index)

C3_temp_df=C3_temp_df.loc[C3_temp_df['CQ'].isin(all_qtrs_YTD),:]
P3_temp_df=P3_temp_df.loc[P3_temp_df['CQ'].isin(all_qtrs_YTD),:]

combined_df=pd.concat([C3_temp_df,P3_temp_df],ignore_index=True)

combined_df.to_excel(output_combined_df, index=False)

In [26]:
combined_df.groupby('Region_')['Standard','Recast','Bulk_Entries'].sum()


DataEntryType,Standard,Recast,Bulk_Entries
Region_,,,
AP,8.847870e+08,3.158282e+07,7.558128e+07
CA,3.153474e+08,4.553346e+06,1.269343e+07
EC,2.551705e+08,2.567859e+06,2.750878e+07
EU,1.793339e+09,7.570853e+06,5.745690e+07
LA,5.188286e+07,3.388832e+05,8.631479e+05
US,1.084796e+09,4.099754e+05,3.558901e+07
ng,5.208741e+04,0.000000e+00,0.000000e+00


In [27]:

ResQ_Prem_Load_df['Entity2']=ResQ_Prem_Load_df['Source'].map(Source_entity_mapping)
ResQ_Prem_Load_df['Reserve Class1']=ResQ_Prem_Load_df[['Region', 'Source','Segment','Line','SubLine1','SubLine2']].apply(lambda x: '\\'.join(x.dropna()), axis=1)
ResQ_Prem_Load_df['Region_']=ResQ_Prem_Load_df['Region'].str[-2:]
ResQ_Prem_Load_df['CQ']=ResQ_Prem_Load_df['Date']
ResQ_Prem_Load_df['Ceded_WP_PQ']=ResQ_Prem_Load_df['Ceded_WP_PQ_Prop']+ResQ_Prem_Load_df['Ceded_WP_PQ_NonProp']
ResQ_Prem_Load_df['Ceded_WP_TQ']=ResQ_Prem_Load_df['Ceded_WP_TQ_Prop']+ResQ_Prem_Load_df['Ceded_WP_TQ_NonProp']
ResQ_Prem_Load_df['Ceded_EP']=ResQ_Prem_Load_df['Ceded_EP_Prop']+ResQ_Prem_Load_df['Ceded_EP_NonProp']
ResQ_Prem_Load_df['Ceded_Unx_EP']=ResQ_Prem_Load_df['Ceded_Unx_EP_Prop']+ResQ_Prem_Load_df['Ceded_Unx_EP_NonProp']
ResQ_Prem_Load_df['Ceded_Unx_Inc']=ResQ_Prem_Load_df['Ceded_Unx_Inc_Prop']+ResQ_Prem_Load_df['Ceded_Unx_Inc_NonProp']
ResQ_Prem_Load_df=ResQ_Prem_Load_df.groupby(['Region_','Entity2','Reserve Class1','CQ'])['Gross_WP_TQ','Ceded_WP_TQ','Gross_EP','Ceded_EP'].sum().reset_index()
ResQ_Prem_Load_df=ResQ_Prem_Load_df.loc[ResQ_Prem_Load_df['CQ'].isin(all_qtrs_YTD),:]
ResQ_Prem_Load_df=pd.merge(ResQ_Prem_Load_df,ResQ_HFM_Mapping_df, how='left', left_on=['Reserve Class1'], right_on=['Reserve Class'] )
ResQ_Prem_Load_df['ExhGrossOS']=0
ResQ_Prem_Load_df['ExhGrossPaid']=0
ResQ_Prem_Load_df['ExhCededOS']=0
ResQ_Prem_Load_df['ExhCededPaid']=0
ResQ_Prem_Load_df.rename(columns={'Gross_WP_TQ': 'ExhGrossWP','Ceded_WP_TQ':'ExhCededWP','Gross_EP':'ExhGrossEP','Ceded_EP':'ExhCededEP'}, inplace=True)
ResQ_Prem_Load_df=ResQ_Prem_Load_df[['Entity2','Region_','HFM','Reserve Class','ExhGrossOS','ExhGrossPaid','ExhCededOS','ExhCededPaid','ExhGrossWP','ExhCededWP','ExhGrossEP','ExhCededEP','CQ']]


import math 
def fixed_cal_qtr2(row):
    qtr_para1=(int(row['Date'][-1])+row['DevMth']/3)/4- math.floor((int(row['Date'][-1])+row['DevMth']/3)/4)
    if qtr_para1*4==0:
        qtr_para2=4
    else:
        qtr_para2=qtr_para1*4
    return str(int(row['Date'][:4])+  int((int(row['Date'][-1])*3+ row['DevMth']-1)/12))+'Q'+ str(int(qtr_para2))

ResQ_Loss_Load_df['CQ']=ResQ_Loss_Load_df.apply(fixed_cal_qtr2, axis=1)

ResQ_Loss_Load_df['Entity2']=ResQ_Loss_Load_df['Source'].map(Source_entity_mapping)
ResQ_Loss_Load_df['Reserve Class1']=ResQ_Loss_Load_df[['Region', 'Source','Segment','Line','Subline1','Subline2']].apply(lambda x: '\\'.join(x.dropna()), axis=1)
ResQ_Loss_Load_df['Region_']=ResQ_Loss_Load_df['Region'].str[-2:]
ResQ_Loss_Load_df['ExhGrossOS']=ResQ_Loss_Load_df['Gross_Inc']-ResQ_Loss_Load_df['Gross_Paid']
ResQ_Loss_Load_df['ExhGrossPaid']=ResQ_Loss_Load_df['Gross_Paid']
ResQ_Loss_Load_df['ExhCededOS']=ResQ_Loss_Load_df['CededNP_Inc']+ResQ_Loss_Load_df['CededP_Inc']-ResQ_Loss_Load_df['CededP_Paid']-ResQ_Loss_Load_df['CededNP_Paid']
ResQ_Loss_Load_df['ExhCededPaid']=ResQ_Loss_Load_df['CededNP_Paid']+ResQ_Loss_Load_df['CededP_Paid']

ResQ_Loss_Load_df=ResQ_Loss_Load_df.groupby(['Region_','Entity2','Reserve Class1','CQ'])['ExhGrossOS','ExhGrossPaid','ExhCededOS','ExhCededPaid'].sum().reset_index()
ResQ_Loss_Load_df=ResQ_Loss_Load_df.loc[ResQ_Loss_Load_df['CQ'].isin(all_qtrs_YTD),:]

ResQ_Loss_Load_df=pd.merge(ResQ_Loss_Load_df,ResQ_HFM_Mapping_df, how='left', left_on=['Reserve Class1'], right_on=['Reserve Class'] )
ResQ_Loss_Load_df['ExhGrossWP']=0
ResQ_Loss_Load_df['ExhCededWP']=0
ResQ_Loss_Load_df['ExhGrossEP']=0
ResQ_Loss_Load_df['ExhCededEP']=0

ResQ_Loss_Load_df=ResQ_Loss_Load_df[['Entity2','Region_','HFM','Reserve Class','ExhGrossOS','ExhGrossPaid','ExhCededOS','ExhCededPaid','ExhGrossWP','ExhCededWP','ExhGrossEP','ExhCededEP','CQ']]


In [28]:
Exhibits_df=pd.merge(ResQ_Loss_Load_df,ResQ_Prem_Load_df,how='outer')
Exhibits_df.to_excel(output_file_Exhibits, index=False)

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:938: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)
C:\Program Files\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:946: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


In [ ]:
Exhibits_df.loc[(Exhibits_df['Region_']=='AP') & (Exhibits_df['HFM']=='Environmental')]


In [ ]:
finance_raw1=pd.merge(finance_raw, SAP_Company_Map_df, how='left', left_on=['SAPCompanyCode'], right_on=['SAP Company Code'] )
finance_raw2=finance_raw1.loc[finance_raw1['GLCompany']==finance_raw1['GLCompany'],:]

In [ ]:
def MJE_0(row):
    if row['DocumentType']!='IG':
        return row['Amount in Local Ccy']
    else:
        return 0

def Genius_0(row):
    if row['DocumentType']=='IG':
        return row['Amount in Local Ccy']
    else:
        return 0

def Test_0(row):
    if row['DocumentType']!=row['DocumentType']:
        return row['Amount in Local Ccy']
    else:
        return 0
    
finance_raw2['MJE_0']=finance_raw2.apply(MJE_0, axis=1)
finance_raw2['Genius_0']=finance_raw2.apply(Genius_0, axis=1)
finance_raw2['Test_0']=finance_raw2.apply(Test_0, axis=1)


In [ ]:
finance_raw2['HFMPrimaryLOB']=finance_raw2['HFMPrimaryLOB'].astype(str)
finance_raw3=finance_raw2.groupby(['SAPGLAccountNumber','AccountGroup','SAPCompanyCode','Local Ccy','DocumentType','HFMPrimaryLOB','SAPLOBCode','SAPRITreatyCode'])['MJE_0','Genius_0','Test_0'].sum().reset_index()
finance_raw3.rename(columns={'MJE_0':'MJE','Genius_0':'Genius','Test_0':'Test'},inplace=True)
finance_raw3.to_excel(output_finance_raw, index=False)

Financials_OrigCurr1=finance_raw2.groupby(['SAPGLAccountNumber','AccountGroup','SAPCompanyCode','Local Ccy','DocumentType','HFMPrimaryLOB','SAPLOBCode','SAPRITreatyCode','Document Ccy'])['Amount in Document Ccy','Amount in Local Ccy'].sum().reset_index()
Financials_OrigCurr2=Financials_OrigCurr1.loc[Financials_OrigCurr1['DocumentType']=='IG',:]
Financials_OrigCurr2.rename(columns={'Amount in Document Ccy':'SumOfAmount in Document Ccy','Amount in Local Ccy':'SumOfAmount in Local Ccy'},inplace=True)
Financials_OrigCurr3=Financials_OrigCurr2.loc[:,['SAPGLAccountNumber','AccountGroup','SAPCompanyCode','DocumentType','HFMPrimaryLOB','SAPLOBCode','SAPRITreatyCode','Document Ccy','SumOfAmount in Document Ccy','Local Ccy','SumOfAmount in Local Ccy']]

Financials_OrigCurr3.to_excel(output_financial_OrigCurr, index=False)